In [2]:
import pandas as pd
import pyodbc
import sqlite3

# iedereen heeft andere servername btw

In [1]:
servername = 'LAPTOP-C1FMPSTV\\SQLEXPRESS01'

# Northwind connectie

In [4]:
northwind_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=NorthWind; Trusted_Connection=yes')
nwCursor = northwind_conn.cursor()
nwCursor

# Adventureworks connectie

In [5]:
adventureworks_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=AdventureWorks2019; Trusted_Connection=yes')
awCursor = adventureworks_conn.cursor()
awCursor

# Aenc connectie

In [6]:
aenc_conn = pyodbc.connect('DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../raw-databases/aenc.accdb;')
aencCursor = aenc_conn.cursor()

table_names = [table_info.table_name for table_info in aencCursor.tables()]

# Print table names
print(table_names)

['MSysAccessStorage', 'MSysACEs', 'MSysComplexColumns', 'MSysIMEXColumns', 'MSysIMEXSpecs', 'MSysNameMap', 'MSysNavPaneGroupCategories', 'MSysNavPaneGroups', 'MSysNavPaneGroupToObjects', 'MSysNavPaneObjectIDs', 'MSysObjects', 'MSysQueries', 'MSysRelationships', 'MSysResources', 'bonus', 'customer', 'department', 'employee', 'product', 'region', 'sales_order', 'sales_order_item', 'state']


In [10]:
aenctest = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
awtest = pd.read_sql_query('SELECT * FROM Person.Address', adventureworks_conn)
nwtest = pd.read_sql_query('SELECT * FROM customers', northwind_conn)

aenctest
print(awtest)
print(nwtest)

C:\Users\sheng\AppData\Local\Temp\ipykernel_125536\2828196301.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  aenctest = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
C:\Users\sheng\AppData\Local\Temp\ipykernel_125536\2828196301.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  awtest = pd.read_sql_query('SELECT * FROM Person.Address', adventureworks_conn)
C:\Users\sheng\AppData\Local\Temp\ipykernel_125536\2828196301.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nwtest = pd.read_sql_query('SELECT * FROM

,id,fname,lname,address,city,state,zip,phone,company_name
0,101,Michaels,Devlin,3114 Pioneer Avenue,Rutherford,NJ,07070,2015558966,The Power Group
1,102,Beth,Reiser,1033 Whippany Road,New York,NY,10154,2125558725,AMF Corp.
2,103,Erin,Niedringhaus,1990 Windsor Street,Paoli,PA,19301,2155556513,Darling Associates
3,104,Meghan,Mason,550 Dundas Street East,Knoxville,TN,37919,6155555463,P.S.C.
4,105,Laura,McCarthy,1210 Highway 36,Carmel,IN,46032,3175558437,Amo & Sons
...,...,...,...,...,...,...,...,...,...
121,552,Janice,O'Toole,654 West Hill,Nashville,TN,37320,6155553689,Greensleeves
122,553,Stevie,Nickolas,77 Recordings Circle,Tacoma,WA,96521,5095551695,It's a Hit!
123,555,Philipe,Fernandez,99 Main Street,Los Angeles,CA,90205,2135554457,Quaker Fashions
124,661,Jennifer,Stutzman,3 Back Pages Lane,Missola,IL,60505,7085556857,Stutzman Advertising


In [13]:
product = pd.read_sql_query('SELECT * FROM Production.Product', adventureworks_conn)
ProductInventory = pd.read_sql_query('SELECT * FROM Production.ProductInventory', adventureworks_conn)
ProductSubCategory = pd.read_sql_query('SELECT * FROM Production.ProductSubCategory', adventureworks_conn)
ProductCategory=pd.read_sql_query('SELECT * FROM Production.ProductCategory', adventureworks_conn)


C:\Users\sheng\AppData\Local\Temp\ipykernel_125536\2817242851.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product = pd.read_sql_query('SELECT * FROM Production.Product', adventureworks_conn)
C:\Users\sheng\AppData\Local\Temp\ipykernel_125536\2817242851.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ProductInventory = pd.read_sql_query('SELECT * FROM Production.ProductInventory', adventureworks_conn)
C:\Users\sheng\AppData\Local\Temp\ipykernel_125536\2817242851.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
 

In [55]:
category = pd.merge(ProductSubCategory,ProductCategory,on="ProductCategoryID",how='inner')
product_quantity = pd.merge(product,ProductInventory,on="ProductID",how="left")
quantity_groupby = product_quantity.groupby(["ProductID","Name","StandardCost","ListPrice"])['Quantity'].sum().reset_index()
product_sumQuantity = pd.merge(product[["ProductID","ProductSubcategoryID"]],quantity_groupby,on="ProductID",how="left")
product_category = pd.merge(category,product_sumQuantity,on="ProductSubcategoryID",how="right")
result = product_category.loc[:,["ProductID","Name","StandardCost","ListPrice","Quantity",'Name_x','Name_y']]
result = result.rename(columns={'Name_x': 'Sub_category'})
result = result.rename(columns={'Name_y': 'Category'})
aw_product = result
aw_product

,ProductID,Name,StandardCost,ListPrice,Quantity,Sub_category,Category
0,1,Adjustable Race,0.0000,0.00,1085.0,NaN,NaN
1,2,Bearing Ball,0.0000,0.00,1109.0,NaN,NaN
2,3,BB Ball Bearing,0.0000,0.00,1352.0,NaN,NaN
3,4,Headset Ball Bearings,0.0000,0.00,1322.0,NaN,NaN
4,316,Blade,0.0000,0.00,1361.0,NaN,NaN
...,...,...,...,...,...,...,...
499,995,ML Bottom Bracket,44.9506,101.24,816.0,Bottom Brackets,Components
500,996,HL Bottom Bracket,53.9416,121.49,970.0,Bottom Brackets,Components
501,997,"Road-750 Black, 44",343.6496,539.99,153.0,Road Bikes,Bikes
502,998,"Road-750 Black, 48",343.6496,539.99,155.0,Road Bikes,Bikes
